In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import scipy.stats as st
import geopy.distance
import requests

import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
from netCDF4 import Dataset
import datetime

%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#Retrieve login credintial for RDA.
from pathlib import Path
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()
session = requests.Session()
session.auth = (userId, pw)

In [ ]:
date=dt.datetime(1993,11,28,6)
lat_range=np.arange(-15,65,0.25)
lon_range=np.arange(360-120,360-40,0.25)  

year=date.year
month=date.month
day=date.day
hour=date.hour

if month<10:
    month_str='0'+str(month)
else:
    month_str=str(month)

if day<10:
    day_str='0'+str(day)
else:
    day_str=str(day)

urlu='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_131_u.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
urlv='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_132_v.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
urlz='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_129_z.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
urlt='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_130_t.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
urlq='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_133_q.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'

store_u = xr.backends.PydapDataStore.open(urlu, session=session)
store_v = xr.backends.PydapDataStore.open(urlv, session=session)
store_z = xr.backends.PydapDataStore.open(urlz, session=session)
store_t = xr.backends.PydapDataStore.open(urlt, session=session)
store_q = xr.backends.PydapDataStore.open(urlq, session=session)

ds_u=xr.open_dataset(store_u)
ds_v=xr.open_dataset(store_v)
ds_z=xr.open_dataset(store_z)
ds_t=xr.open_dataset(store_t)
ds_q=xr.open_dataset(store_q)

print(ds_u)

u=ds_u.U.sel(latitude=lat_range,longitude=lon_range)
v=ds_v.V.sel(latitude=lat_range,longitude=lon_range)
z=ds_z.Z.sel(latitude=lat_range,longitude=lon_range)
t=ds_t.T.sel(latitude=lat_range,longitude=lon_range)
q=ds_q.Q.sel(latitude=lat_range,longitude=lon_range)
print(u)

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
# Grab lat/lon values (GFS will be 1D)
#lats = ds_u.latitude.sel(latitude=lat_slice).values
#lons = ds_u.longitude.sel(longitude=lon_slice).values

lats = u['latitude'].metpy.unit_array
lons = u['longitude'].metpy.unit_array

vtime=date
hght_850 = z.metpy.sel(vertical=700 * units.hPa,time=vtime)
tmpk_850 = t.metpy.sel(vertical=700 * units.hPa,time=vtime)
uwnd_850 = u.metpy.sel(vertical=700 * units.hPa,time=vtime)
vwnd_850 = v.metpy.sel(vertical=700 * units.hPa,time=vtime)

# Grab data and smooth using a nine-point filter applied 50 times (edit: 150) to grab the synoptic signal
level = 700 * units.hPa
hght_850 = mpcalc.smooth_n_point(hght_850.squeeze(), 9, 150)
tmpk_850 = mpcalc.smooth_n_point(tmpk_850.squeeze(), 9, 150)#25
uwnd_850 = mpcalc.smooth_n_point(uwnd_850.squeeze(), 9, 150)
vwnd_850 = mpcalc.smooth_n_point(vwnd_850.squeeze(), 9, 150)

# Convert temperatures to degree Celsius for plotting purposes
tmpc_850 = tmpk_850.to('degC')

In [ ]:
# Compute grid spacings for data
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

# Compute the Q-vector components
uqvect, vqvect = mpcalc.q_vector(uwnd_850, vwnd_850, tmpk_850, 700*units.hPa, dx, dy)

# Compute the divergence of the Q-vectors calculated above
q_div = -2*mpcalc.divergence(uqvect, vqvect, dx, dy, dim_order='yx')

In [ ]:
hght_850_vals=hght_850.m/100.
print(hght_850_vals)

In [ ]:
# Set the map projection (how the data will be displayed)
mapcrs = ccrs.LambertConformal(
    central_longitude=-75, central_latitude=40, standard_parallels=(30, 60))

# Set the data project (GFS is lat/lon format)
datacrs = ccrs.PlateCarree()

# Start the figure and set an extent to only display a smaller graphics area
fig = plt.figure(1, figsize=(12, 14))
ax = plt.subplot(111, projection=mapcrs)
ax.set_extent([-95, -55, 25, 55], ccrs.PlateCarree())

# Add map features to plot coastlines and state boundaries
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))

if 1==1:
    if 1==1:
        if 1==0:
            fig.canvas.draw()

            # Define gridline locations and draw the lines using cartopy's built-in gridliner:
            xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
            yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
            ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
            ax.tick_params(labelsize=24)
            # Label the end-points of the gridlines using the custom tick makers:
            ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
            ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
            lambert_xticks(ax, xticks)
            lambert_yticks(ax, yticks)
            
# Plot 850-hPa Q-Vector Divergence and scale
clevs_850_tmpc = np.arange(-40, 41, 2)
clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
cf = ax.contourf(lons, lats, q_div*1e18*-1., clevs_qdiv, cmap=plt.cm.bwr,
                 extend='both', transform=datacrs)
cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True,
                  ticks=clevs_qdiv)
cb.set_label('Q-Vector Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)',fontsize=17)
cb.ax.tick_params(labelsize=15)

# Plot 850-hPa Temperatures
csf = ax.contour(lons, lats, tmpc_850, clevs_850_tmpc, colors='grey',
                 linestyles='dashed', transform=datacrs)
plt.clabel(csf, fmt='%d',fontsize=14)

# Plot 850-hPa Geopotential Heights
clevs_850_hght = np.arange(0, 400, 3)
cs = ax.contour(lons, lats, hght_850_vals, clevs_850_hght, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%d',fontsize=15)

# Plot 850-hPa Q-vectors, scale to get nice sized arrows
wind_slice = (slice(None, None, 5), slice(None, None, 5))
print(lons[wind_slice[0]])
print(lons[wind_slice[0]].magnitude)
print(uqvect[wind_slice].m)
ax.quiver(lons[wind_slice[0]].magnitude, lats[wind_slice[1]].magnitude,
          uqvect[wind_slice].m,
          vqvect[wind_slice].m,
          pivot='mid', color='black',
          scale=1e-11*0.5, scale_units='inches',
          transform=datacrs)

# Add some titles
plt.title('700-hPa ERA5 Geo. Heights (dm), Temp (C),'
          ' and Q-Vectors (m$^2$ kg$^{-1}$ s$^{-1}$)', loc='left',fontsize=13)
plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=13)
#plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
ax.text(-88,57.6,'Q-vector divergence',transform=ccrs.PlateCarree(),fontsize=30)
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig(dir+'q_vector_example6_1a.png')

In [ ]:
urlmfc='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.vinteg/'+str(year)+month_str+'/e5.oper.an.vinteg.162_084_viwvd.ll025sc.'+str(year)+month_str+'0100_'+str(year)+month_str+'3023.nc'
#https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.vinteg/201710/e5.oper.an.vinteg.162_084_viwvd.ll025sc.2017100100_2017103123.nc

store_mfc = xr.backends.PydapDataStore.open(urlmfc, session=session)

ds_mfc=xr.open_dataset(store_mfc)

print(ds_mfc)

mfc=ds_mfc.VIWVD.sel(time=date,latitude=lat_range,longitude=lon_range)
print(mfc)

In [ ]:
# Set the map projection (how the data will be displayed)
mapcrs = ccrs.LambertConformal(
    central_longitude=-75, central_latitude=40, standard_parallels=(30, 60))

# Set the data project (GFS is lat/lon format)
datacrs = ccrs.PlateCarree()

# Start the figure and set an extent to only display a smaller graphics area
fig = plt.figure(1, figsize=(12, 12))
ax = plt.subplot(111, projection=mapcrs)
ax.set_extent([-95, -55, 25, 55], ccrs.PlateCarree())

# Add map features to plot coastlines and state boundaries
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))

# Plot 850-hPa Q-Vector Divergence and scale
#clevs_850_tmpc = np.arange(-40, 41, 2)
clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
cf = ax.contourf(lons, lats, mfc*1e4, clevs_qdiv, cmap=plt.cm.bwr,
                 extend='both', transform=datacrs)
cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True,
                  ticks=clevs_qdiv,fraction=0.046)
cb.set_label('Moisture Flux Div. (*10$^{4}$ kg m$^{-2}$ s$^{-1}$)',fontsize=17)
cb.ax.tick_params(labelsize=15)

# Plot 850-hPa Temperatures
#csf = ax.contour(lons, lats, tmpc_850, clevs_850_tmpc, colors='grey',linestyles='dashed', transform=datacrs)
#plt.clabel(csf, fmt='%d',fontsize=14)

# Plot 850-hPa Geopotential Heights
#clevs_850_hght = np.arange(0, 400, 3)
#cs = ax.contour(lons, lats, hght_850_vals, clevs_850_hght, colors='black', transform=datacrs)
#plt.clabel(cs, fmt='%d',fontsize=15)

# Plot 850-hPa Q-vectors, scale to get nice sized arrows
if 1==0:
    wind_slice = (slice(None, None, 5), slice(None, None, 5))
    print(lons[wind_slice[0]])
    print(lons[wind_slice[0]].magnitude)
    print(uqvect[wind_slice].m)
    ax.quiver(lons[wind_slice[0]].magnitude, lats[wind_slice[1]].magnitude,
              uqvect[wind_slice].m,
              vqvect[wind_slice].m,
              pivot='mid', color='black',
              scale=1e-2*0.5, scale_units='inches',
              transform=datacrs)

# Add some titles
plt.title('ERA5 integrated moisture flux divergence (kg m$^{-2}$ s$^{-1}$)', loc='left',fontsize=13)
plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=13)
#plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
ax.text(-90,57.6,'Moisture flux divergence',transform=ccrs.PlateCarree(),fontsize=30)
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig(dir+'q_vector_example6_2a.png')

In [ ]:
dz2=xr.open_dataset(dir+'ivt_basics_neusa_90.nc')
longitude_sub=dz2.longitude.values
ds2=xr.open_dataset(dir+'ivt_basics_neusa_90.nc')
ds2['longitude']=longitude_sub
#ivt_threshold=528.6942100292907
ivt_threshold=ds2['ivt_threshold']
ivt_mag=ds2['ivt_mag']
ivtx=ds2['ivtx']
ivty=ds2['ivty']
lons_full=ivt_mag.longitude.values
lats_full=ivt_mag.latitude.values
print(ivt_mag)

lat_range=np.arange(0,60,0.25)
lon_range=np.arange(-140,-110,0.25)

if 1==1:
    ds3=xr.open_dataset(dir+'era_5_fall_u_850_neusa.nc')
    u_850=ds3['u']#.sel(latitude=lat_range,longitude=lon_range)
    ds4=xr.open_dataset(dir+'era_5_fall_v_850_neusa.nc')
    v_850=ds4['v']#.sel(latitude=lat_range,longitude=lon_range)
    ds5=xr.open_dataset(dir+'era_5_fall_mslp_neusa.nc')
    print(ds5)
    slp=ds5['msl']#.sel(latitude=lat_range,longitude=lon_range)/100.

    ds6=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
    ds6['longitude']=ds3.longitude.values
    print(ds6)
    slp_2=ds6['mslp']#.sel(latitude=lat_range,longitude=lon_range)
    ds7=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
    ds7['longitude']=ds3.longitude.values
    u_850_2=ds7['u']#.sel(latitude=lat_range,longitude=lon_range)
    v_850_2=ds7['v']#.sel(latitude=lat_range,longitude=lon_range)

In [ ]:
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(dates)

ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats1=ds1['lats']#use these lists for plotting
lons1=ds1['lons']+360.
stations1=ds1['stations']
thresholds=ds1['thresholds_annual_99']/10.

In [ ]:
ds6=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_na_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

In [ ]:
dsa=xr.open_dataset(dir+'neusa_ar_climo_fall.nc')
ar=dsa.ar
tc_trop=dsa.tc_trop
tc_nontrop=dsa.tc_nontrop
labeled=dsa.labeled

In [ ]:
#ivt_threshold=528.6942100292907
plt.close()
dates_unique=[date]
import matplotlib as mpl
lats2=ds2.latitude.values.tolist()
lons2=ds2.longitude.values.tolist()
zipped=list(zip(dates,lats,lons,obs))
zipped_thresh=list(zip(thresholds,lats1,lons1))
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
mapcrs = ccrs.LambertConformal(
    central_longitude=-75, central_latitude=40, standard_parallels=(30, 60))
count_fig=0
count_file=0
for d in range(len(dates_unique)):
    print(d)
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    select_date=[x for x in zipped if x[0]==date]
    date_midday=date+dt.timedelta(hours=12)

    date_range=[date_midday]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT, AR, EP comparison
    
    labeled_four=labeled.sel(time=date_range)
    points_closest_list=[]
    obs_extreme_list=[]
    lats_extreme_list=[]
    lons_extreme_list=[]
    four_labeled_rain=xr.zeros_like(labeled_four)
    for i in range(len(lats1)):
        lat=lats1[i]
        lon=lons1[i]
        select_station=[x for x in select_date if x[1]==lat and x[2]==lon]
        select_thresh=[x for x in zipped_thresh if x[1]==lat and x[2]==lon]
        if len(select_station)>1:
            print('ERROR')
        if len(select_station)>0:
            obs_1=select_station[0][-1]
            thresh=select_thresh[0][0].values
            if obs_1>=thresh:
                obs_extreme_list.append(obs_1)
                lats_extreme_list.append(lat)
                lons_extreme_list.append(lon)
                

    #for date in date_range:
    if 1==1:
        print(date)
        #date=pd.to_datetime(date)
        
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        
        ivt_mag_current=ivt_mag.sel(time=date)
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)

        if month==12 and day==1:
            u_850_select=u_850_2.sel(time=date)
            v_850_select=v_850_2.sel(time=date)
            slp_select=slp_2.sel(time=date)
        else:
            u_850_select=u_850.sel(time=date)
            v_850_select=v_850.sel(time=date)
            slp_select=slp.sel(time=date)

        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_250=ivt_mag_current.where(ivt_mag_current>=250.)
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        ar_select=ar.sel(time=date)
        
        if 1==1:
            fig = plt.figure(figsize=(12,14))
            ax=plt.subplot(1,1,1,projection=mapcrs)
        
        check_lat_tc=lat_tc.where(years_tc==year,drop=True)
        check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
        check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
        #TS,SS,EX,ET,
        check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
        check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)
        
        check_lat_tc_trop_list=[]
        for i in range(check_lat_tc_trop.storm.size):
            check_lat_tc_trop_values=check_lat_tc_trop[i,:].values.tolist()
            check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
            if len(check_lat_tc_trop_values)>0:
                check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                check_lat_tc_trop_list.append(check_lat_tc_trop_value)
                
        check_lat_tc_nontrop_list=[]
        for i in range(check_lat_tc_nontrop.storm.size):
            check_lat_tc_nontrop_values=check_lat_tc_nontrop[i,:].values.tolist()
            check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
            if len(check_lat_tc_nontrop_values)>0:
                check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)
        
        check_lon_tc=lon_tc.where(years_tc==year,drop=True)
        check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
        check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
        check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
        check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)
        
        check_lon_tc_trop_list=[]
        for i in range(check_lon_tc_trop.storm.size):
            check_lon_tc_trop_values=check_lon_tc_trop[i,:].values.tolist()
            check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
            if len(check_lon_tc_trop_values)>0:
                check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                check_lon_tc_trop_list.append(check_lon_tc_trop_value)
                
        check_lon_tc_nontrop_list=[]
        for i in range(check_lon_tc_nontrop.storm.size):
            check_lon_tc_nontrop_values=check_lon_tc_nontrop[i,:].values.tolist()
            check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
            if len(check_lon_tc_nontrop_values)>0:
                check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                print((lat_tc_single,lon_tc_single))
                if 23<=lat_tc_single<=50 and -90<=lon_tc_single<=-50:
                    ax.plot(lon_tc_single,lat_tc_single,marker='*',color='r',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                for j in range(len(lats_extreme_list)):
                    lat_rain=lats_extreme_list[j]
                    lon_rain=lons_extreme_list[j]
                    distance=geopy.distance.distance((lat_tc_single,lon_tc_single),(lat_rain,lon_rain)).km
                    if distance<min_distance_tc:
                        min_distance_tc=distance
                    if distance<=500:
                        if lat_rain not in tc_trop_lat_list:
                            tc_trop_lat_list.append(lat_rain)
                            tc_trop_lon_list.append(lon_rain)
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                print((lat_tc_single,lon_tc_single))
                if 23<=lat_tc_single<=50 and -90<=lon_tc_single<=-50:
                    ax.plot(lon_tc_single,lat_tc_single,marker='*',color='saddlebrown',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                for j in range(len(lats_extreme_list)):
                    lat_rain=lats_extreme_list[j]
                    lon_rain=lons_extreme_list[j]
                    distance=geopy.distance.distance((lat_tc_single,lon_tc_single),(lat_rain,lon_rain)).km
                    if distance<min_distance_tc:
                        min_distance_tc=distance
                    if distance<=500:
                        if lat_rain not in tc_nontrop_lat_list:
                            tc_nontrop_lat_list.append(lat_rain)
                            tc_nontrop_lon_list.append(lon_rain)
        if 1==1:             
            ax.coastlines(resolution='10m')
            ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            ax.set_extent([-95, -55, 25, 55], ccrs.PlateCarree())
            
            if 1==0:
                # *must* call draw in order to get the axis boundary used to add ticks:
                fig.canvas.draw()

                # Define gridline locations and draw the lines using cartopy's built-in gridliner:
                xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
                yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
                ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
                ax.tick_params(labelsize=24)
                # Label the end-points of the gridlines using the custom tick makers:
                ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
                ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
                lambert_xticks(ax, xticks)
                lambert_yticks(ax, yticks)

            cax1=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250,levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5)
            
            cbar = plt.colorbar(cax1, orientation='horizontal', pad=0, aspect=50,fraction=0.046)
            cbar.set_label('kg/m/s',fontsize=20,labelpad=10)
            cbar.ax.tick_params(labelsize=20)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,linewidths=1.0,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
            plt.clabel(c,fmt='%.00f',fontsize=15)
            X=6
            q=ax.quiver(u_850_select.longitude[0::X].values,u_850_select.latitude[0::X].values,u_850_select[0::X,0::X].values,v_850_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=30,width=0.025,alpha=0.5)
            ax.quiverkey(q,0.67,1.01,10,'10 m/s',fontproperties={'size':13})

            cax21=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250.where(ar_select==1),levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5,label='Identified AR')
            cax2=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ar_select,[0,1],colors='b',transform=ccrs.PlateCarree(),hatches=[None,'.'],alpha=0.1,label='Identified AR',zorder=20)

            mpl.rcParams['hatch.linewidth']=0.01
            vtime=date
            plt.title('ERA5 850-hPa winds (m/s), MSLP (hPa), and IVT (kg m$^{-1}$ s$^{-1}$)',loc='left',fontsize=13)
            plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=13)
            #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
            ax.text(-84,57.6,'AR detection',transform=ccrs.PlateCarree(),fontsize=30)
            plt.tight_layout()
            plt.show()

In [ ]:
fig.savefig(dir+'q_vector_example6_3a.png')

In [ ]:
vtime=date
hght_250 = z.metpy.sel(vertical=250 * units.hPa,time=vtime)
tmpk_250 = t.metpy.sel(vertical=250 * units.hPa,time=vtime)
uwnd_250 = u.metpy.sel(vertical=250 * units.hPa,time=vtime)
vwnd_250 = v.metpy.sel(vertical=250 * units.hPa,time=vtime)

In [ ]:
speed_250=np.sqrt(uwnd_250**2+vwnd_250**2)

In [ ]:
slp_select=slp.sel(time=date)
thickness=z.sel(time=date,level=500)-z.sel(time=date,level=1000)

In [ ]:
print(speed_250.max().values)

In [ ]:
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
mapcrs = ccrs.LambertConformal(central_longitude=-75, central_latitude=40, standard_parallels=(30, 60))

if 1==1:
    if 1==1:
        if 1==1:
            fig = plt.figure(figsize=(12,14))
            ax=plt.subplot(1,1,1,projection=mapcrs)
            ax.coastlines(resolution='10m')
            ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            #ax.set_extent([-95, -55, 20, 50], ccrs.PlateCarree())
            ax.set_extent([-95, -55, 25, 55], ccrs.PlateCarree())
            
            if 1==0:
                # *must* call draw in order to get the axis boundary used to add ticks:
                fig.canvas.draw()

                # Define gridline locations and draw the lines using cartopy's built-in gridliner:
                xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
                yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
                ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
                ax.tick_params(labelsize=24)
                # Label the end-points of the gridlines using the custom tick makers:
                ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
                ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
                lambert_xticks(ax, xticks)
                lambert_yticks(ax, yticks)

            #cax=ax.scatter(lons_extreme_list,lats_extreme_list,s=150,c=obs_extreme_list,transform=ccrs.PlateCarree(),vmin=0,vmax=120,cmap=plt.cm.RdYlGn_r,zorder=10)
            cax1=ax.contourf(speed_250.longitude,speed_250.latitude,speed_250.where(speed_250>=30.),levels=np.arange(30,120,10),transform=ccrs.PlateCarree(),colors=['skyblue','deepskyblue','dodgerblue','lightpink','lightcoral','tomato','deeppink','lightgray','whitesmoke'],alpha=0.5)
            
            cbar = plt.colorbar(cax1, orientation='horizontal', pad=0, aspect=50,fraction=0.046,ticks=[40,50,60,70,80,90,100])
            cbar.set_label('m/s',fontsize=20,labelpad=10)
            cbar.ax.tick_params(labelsize=20)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,linewidths=1.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
            plt.clabel(c,fmt='%.00f',fontsize=15)
            
            c1=ax.contour(thickness.longitude,thickness.latitude,thickness/100.,linewidths=1.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
            plt.clabel(c1,fmt='%.00f',fontsize=15)
            
            #ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=36)
            
            vtime=date
            plt.title('ERA5 250-hPa wind speed (m/s), MSLP (hPa), and 1000-500-hPa thickness (dm)',loc='left',fontsize=13)
            plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=13)
            #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
            ax.text(-83,57.6,'Upper-level',transform=ccrs.PlateCarree(),fontsize=30)
            plt.tight_layout()
            plt.show()

In [ ]:
fig.savefig(dir+'q_vector_example6_4a.png')

In [ ]:
vtime=date
uwnd_700 = u.metpy.sel(vertical=700 * units.hPa,time=vtime)
vwnd_700 = v.metpy.sel(vertical=700 * units.hPa,time=vtime)
speed_700=np.sqrt(uwnd_700**2+vwnd_700**2)

In [ ]:
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
mapcrs = ccrs.LambertConformal(central_longitude=-75, central_latitude=40, standard_parallels=(30, 60))

if 1==1:
    if 1==1:
        if 1==1:
            fig = plt.figure(figsize=(12,14))
            ax=plt.subplot(1,1,1,projection=mapcrs)
            ax.coastlines(resolution='10m')
            ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            #ax.set_extent([-95, -55, 20, 50], ccrs.PlateCarree())
            ax.set_extent([-95, -55, 25, 55], ccrs.PlateCarree())
            
            if 1==0:
                # *must* call draw in order to get the axis boundary used to add ticks:
                fig.canvas.draw()

                # Define gridline locations and draw the lines using cartopy's built-in gridliner:
                xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
                yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
                ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
                ax.tick_params(labelsize=24)
                # Label the end-points of the gridlines using the custom tick makers:
                ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
                ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
                lambert_xticks(ax, xticks)
                lambert_yticks(ax, yticks)

            #cax=ax.scatter(lons_extreme_list,lats_extreme_list,s=150,c=obs_extreme_list,transform=ccrs.PlateCarree(),vmin=0,vmax=120,cmap=plt.cm.RdYlGn_r,zorder=10)
            cax1=ax.contourf(speed_700.longitude,speed_700.latitude,speed_700.where(speed_700>=30.),levels=np.arange(30,120,10),transform=ccrs.PlateCarree(),colors=['skyblue','deepskyblue','dodgerblue','lightpink','lightcoral','tomato','deeppink','lightgray','whitesmoke'],alpha=0.5)
            
            cbar = plt.colorbar(cax1, orientation='horizontal', pad=0, aspect=50,fraction=0.046,ticks=[40,50,60,70,80,90,100])
            cbar.set_label('m/s',fontsize=20,labelpad=10)
            cbar.ax.tick_params(labelsize=20)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,linewidths=1.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4),alpha=0.5)
            plt.clabel(c,fmt='%.00f',fontsize=15)
            
            c1=ax.contour(thickness.longitude,thickness.latitude,thickness/100.,linewidths=1.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
            plt.clabel(c1,fmt='%.00f',fontsize=15)
            
            #ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=36)
            
            X=6
            q=ax.quiver(uwnd_700.longitude[0::X].values,uwnd_700.latitude[0::X].values,uwnd_700[0::X,0::X].values,vwnd_700[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=90,width=0.025,alpha=1.0)
            ax.quiverkey(q,0.69,1.01,30,'30 m/s',fontproperties={'size':13})


            vtime=date
            plt.title('ERA5 700-hPa wind speed (m/s), MSLP (hPa), and 1000-500-hPa thickness (dm)',loc='left',fontsize=13)
            plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=13)
            #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
            ax.text(-85.5,57.6,'700 hPa winds',transform=ccrs.PlateCarree(),fontsize=30)
            plt.tight_layout()
            plt.show()

In [ ]:
fig.savefig(dir+'q_vector_example6_5a.png')